In [1]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine, inspect

In [2]:
# read in vgsales kaggle csv
df = pd.read_csv("Resources/vgsales.csv")

In [11]:
# remove the second index column
df.reset_index(drop = True)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [12]:
# list the games that we need to pulll based on the Business Insider web scrape
games = ['World of Goo', 'Metal Gear Solid V: The Phantom Pain', 'Portal 2', 'Persona 5 Royal', 'Madden NFL 2003', 'The Last of Us', "The Legend of Zelda: Majora's Mask", 'The Legend of Zelda: A Link to the Past', 'Halo 2', 'Gran Turismo 3: A-Spec', 'Red Dead Redemption', "The Legend of Zelda: Collector's Edition", 'LittleBigPlanet', 'Grand Theft Auto: Vice City', 'Grand Theft Auto: San Andreas', "Baldur's Gate II: Shadows of Amn", 'Metal Gear Solid 2: Sons of Liberty', 'Gran Turismo', 'The Legend of Zelda: The Wind Waker', 'Half-Life', 'The Elder Scrolls V: Skyrim', 'The Legend of Zelda: Twilight Princess', 'The House in Fata Morgana - Dreams of the Reve', 'Mass Effect 2', 'Tekken 3', 'Batman: Arkham City', 'The Orange Box', 'Resident Evil 4', 'Uncharted 2: Among Thieves', 'GoldenEye 007', 'BioShock', 'Half-Life 2', 'NFL 2K1', 'Elden Ring', 'Halo: Combat Evolved', 'Super Mario Odyssey', 'Grand Theft Auto III', 'Metroid Prime', 'Perfect Dark', "Tony Hawk's Pro Skater 3", 'The Legend of Zelda: Breath of the Wild', 'Disco Elysium: The Final Cut', 'Grand Theft Auto V', 'Red Dead Redemption 2', 'Super Mario Galaxy 2', 'Super Mario Galaxy','SoulCalibur', 'Grand Theft Auto IV', "Tony Hawk's Pro Skater 2", 'The Legend of Zelda: Ocarina of Time']


In [13]:
# pull the games from the CSV using the list and drop un-needed columns
games_df = df.loc[df['Name'].isin(games), ['Name', 'Platform', 'Year', 'Genre', 'Global_Sales']].reset_index(drop = True)


In [14]:
games_df

,Name,Platform,Year,Genre,Global_Sales
0,Grand Theft Auto V,PS3,2013.0,Action,21.40
1,Grand Theft Auto: San Andreas,PS2,2004.0,Action,20.81
2,Grand Theft Auto V,X360,2013.0,Action,16.38
3,Grand Theft Auto: Vice City,PS2,2002.0,Action,16.15
4,Gran Turismo 3: A-Spec,PS2,2001.0,Racing,14.98
...,...,...,...,...,...
88,Grand Theft Auto: Vice City,PC,2003.0,Action,0.04
89,Halo: Combat Evolved,PC,2003.0,Shooter,0.04
90,World of Goo,PC,2008.0,Puzzle,0.04
91,Resident Evil 4,PC,2007.0,Action,0.03


In [15]:
# read in the BI web scraped CSV
df2 = pd.read_csv("Resources/business_i.csv")

In [16]:
df2

,Name,Critic Score,User Score
0,World of Goo,94,8.8
1,Metal Gear Solid V: The Phantom Pain,95,7.4
2,Portal 2,95,8.4
3,Persona 5 Royal,95,8.4
4,Madden NFL 2003,95,7.5
5,The Last of Us,95,9.2
6,The Legend of Zelda: Majora's Mask,95,9.2
7,The Legend of Zelda: A Link to the Past,95,9.0
8,Halo 2,95,8.2
9,Gran Turismo 3: A-Spec,95,8.5


In [17]:
# merge the BI CSV with the cleaned vgsales CSV
df_merge = pd.merge(games_df, df2, on='Name')
df_merge.to_csv('game_sales.csv', index = False)

In [18]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          93 non-null     object 
 1   Platform      93 non-null     object 
 2   Year          93 non-null     float64
 3   Genre         93 non-null     object 
 4   Global_Sales  93 non-null     float64
 5   Critic Score  93 non-null     int64  
 6   User Score    93 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 5.8+ KB


In [19]:
# Pull the platforms columns
platforms = df_merge.copy()
platforms

,Name,Platform,Year,Genre,Global_Sales,Critic Score,User Score
0,Grand Theft Auto V,PS3,2013.0,Action,21.40,96,7.8
1,Grand Theft Auto V,X360,2013.0,Action,16.38,96,7.8
2,Grand Theft Auto V,PS4,2014.0,Action,11.98,96,7.8
3,Grand Theft Auto V,XOne,2014.0,Action,5.08,96,7.8
4,Grand Theft Auto V,PC,2015.0,Action,1.08,96,7.8
...,...,...,...,...,...,...,...
88,The Orange Box,PS3,2007.0,Shooter,1.08,96,9.2
89,The Orange Box,PC,2007.0,Shooter,0.14,96,9.2
90,NFL 2K1,DC,2000.0,Sports,1.09,97,6.3
91,SoulCalibur,DC,1999.0,Fighting,0.34,98,8.6


In [20]:
# remove the duplicate consoles and save off to CSV
platform_df = platforms.Platform
plat = platform_df.drop_duplicates(keep='first', inplace=False).reset_index(drop = True).to_csv("platform.csv", index = False)
plat

In [44]:
# isolate the genre column from the merge and save off as CSV
genres = df_merge.Genre.drop_duplicates(keep = 'first', inplace = False).reset_index(drop = True)
genres

0          Action
1          Racing
2        Platform
3    Role-Playing
4         Shooter
5        Fighting
6          Sports
7          Puzzle
Name: Genre, dtype: object

In [34]:
genres.to_csv("genres.csv", index = False)

In [3]:
# connect to postgres (pgadmin) server
HOSTNAME = "35.224.199.242"
PORT = 5432
DATABASE = "postgres"
USERNAME = 'postgres'
PASSWORD = "group2"

In [4]:
rds_connection_string = "postgres:group2@35.224.199.242:5432/postgres"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [5]:
# test engine to see if it works
engine.table_names()

/var/folders/6m/cq811jws6_76rhtzmz7w7m9c0000gn/T/ipykernel_83003/1339351410.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['GameGenres',
 'GamesReleasedPlatforms',
 'videogames',
 'genre_junc',
 'platform_junc',
 'genre_id',
 'platforms',
 'all_games']

In [40]:
# import genre csv into the genres table
genres = pd.read_csv("genres.csv")

In [45]:
genres.to_sql(name='genre', con=engine, if_exists='append', index=False)

In [46]:
pd.read_sql_query('select * from genre', con=engine).head()

,id,Genre,last_updated
0,1,Puzzle,2022-03-22 17:46:06.816234
1,2,Action,2022-03-22 17:46:06.816234
2,3,Shooter,2022-03-22 17:46:06.816234
3,4,Sports,2022-03-22 17:46:06.816234
4,5,Racing,2022-03-22 17:46:06.816234


In [25]:
pd.read_sql_query('select * from videogames', con=engine).head()

,id,name,platform,year,genre,global_sales,critic_score,user_score,last_updated
0,1,Grand Theft Auto V,PS3,2013.0,Action,21.40,96.0,7.8,2022-03-22 02:37:36.232415
1,2,Grand Theft Auto V,X360,2013.0,Action,16.38,96.0,7.8,2022-03-22 02:37:36.232415
2,3,Grand Theft Auto V,PS4,2014.0,Action,11.98,96.0,7.8,2022-03-22 02:37:36.232415
3,4,Grand Theft Auto V,XOne,2014.0,Action,5.08,96.0,7.8,2022-03-22 02:37:36.232415
4,5,Grand Theft Auto V,PC,2015.0,Action,1.08,96.0,7.8,2022-03-22 02:37:36.232415


In [27]:
pd.read_sql_query('select * from platforms', con=engine).head()

,id,platform,last_updated
0,1,PS3,2022-03-22 02:48:42.762126
1,2,X360,2022-03-22 02:48:42.762126
2,3,PS4,2022-03-22 02:48:42.762126
3,4,XOne,2022-03-22 02:48:42.762126
4,5,PC,2022-03-22 02:48:42.762126
